In [1]:
import fastf1 as ff1
import pandas as pd
from datetime import datetime
import numpy as np
from fastf1.ergast import Ergast
import requests
from functools import lru_cache

In [2]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

In [3]:
ff1.Cache.enable_cache('data')

In [4]:
def get_driver_max_throttle_ratio(session, driver, max_throttle_threshold=98, season=None, session_name=None):
    """
    Get the max throttle ratio for the fastest lap in the session for a selected driver.

    Parameters:
    - session: FastF1 loaded session object (must be .load()-ed)
    - driver: abbreviated driver name (e.g., 'VER')
    - max_throttle_threshold: threshold for detecting 'max throttle' zones (default: 98%)
    - season: (optional) int year
    - session_name: (optional) string like 'FP1', used for session_uid

    Returns:
    - full_throttle: DataFrame with driver metrics and session_uid
    - missing_info: DataFrame with fallback in case of telemetry issues
    """

    gp_name = session.event['EventName']
    location = session.event['Location']

    try:
        fastest_driver = session.laps.pick_driver(driver).pick_fastest()
        telemetry = fastest_driver.get_telemetry().add_distance()

        # Merge weather info
        telemetry = pd.merge_asof(
            telemetry,
            session.weather_data[['Time', 'Rainfall', 'TrackTemp', 'AirTemp']],
            left_on='SessionTime',
            right_on='Time'
        )

        # Calculate heavy braking zones
        telemetry['delta_speed'] = telemetry['Speed'].diff()
        heavy_brakes = telemetry['delta_speed'] < -30  # threshold in km/h
        braking_events = heavy_brakes.sum()

        # Identify throttle segments
        telemetry['nextThrottle'] = telemetry.Throttle.shift(-1)
        telemetry['previousThrottle'] = telemetry.Throttle.shift(1)

        throttle_points = telemetry.loc[
            (telemetry.Throttle >= max_throttle_threshold) &
            (
                (telemetry.nextThrottle < max_throttle_threshold) |
                (telemetry.previousThrottle < max_throttle_threshold) |
                (telemetry.index.isin([telemetry.index[0], telemetry.index[-1]]))
            )
        ].copy()

        throttle_points['FTRelative'] = throttle_points.RelativeDistance.diff().fillna(0)

        # Calculate full throttle ratio
        ratio = throttle_points.loc[
            (throttle_points.nextThrottle < max_throttle_threshold) |
            (throttle_points.nextThrottle.isna())
        ]['FTRelative'].sum()

        # Prepare result row
        result = pd.DataFrame([{
            'grand_prix': gp_name,
            'location': location,
            'driver': driver,
            'ratio': ratio,
            'compound': fastest_driver['Compound'],
            'tyre_age': fastest_driver['TyreLife'],
            'is_fresh_tyre': fastest_driver['FreshTyre'],
            'avg_rainfall': telemetry['Rainfall'].mean(),
            'avg_track_temp': telemetry['TrackTemp'].mean(),
            'avg_air_temp': telemetry['AirTemp'].mean(),
            'braking_events': braking_events,
            'session_uid': f"{season}_{location}_{session_name}" if season and session_name else None
        }])

        return result, None

    except Exception:
        # Telemetry missing or failed to load
        missing = pd.DataFrame([{
            'grand_prix': gp_name,
            'location': location,
            'driver': driver,
            'session_uid': f"{season}_{location}_{session_name}" if season and session_name else None
        }])
        return None, missing


In [5]:
def get_all_drivers_throttle_input(session, season=None, session_name=None):
    """
    Get the max throttle ratio and braking event data for all drivers in a session.

    Parameters:
    - session: loaded FastF1 session
    - season: optional, used to generate session_uid
    - session_name: optional, like 'FP1', used to generate session_uid

    Returns:
    - correct_readings: DataFrame with all valid telemetry-derived data
    - missing_info: DataFrame with drivers who had missing or invalid data
    """
    drivers = pd.unique(session.laps['Driver'])
    gp_name = session.event['EventName']
    location = session.event['Location']

    throttle_data = []
    missing_data = []

    for driver in drivers:
        result, missing = get_driver_max_throttle_ratio(
            session, driver,
            season=season,
            session_name=session_name
        )
        if result is not None:
            throttle_data.append(result)
        if missing is not None:
            missing_data.append(missing)

    # Combine all gathered rows
    full_throttle = pd.concat(throttle_data, ignore_index=True) if throttle_data else pd.DataFrame()
    missing_info = pd.concat(missing_data, ignore_index=True) if missing_data else pd.DataFrame()

    # Remove outliers: throttle ratio must be between 0.4 and 0.85
    full_throttle.loc[(full_throttle.ratio > 0.85) | (full_throttle.ratio < 0.4), 'ratio'] = np.NaN

    correct_readings = full_throttle.dropna(subset=['ratio'])
    incorrect_readings = full_throttle[full_throttle.ratio.isna()]

    if not incorrect_readings.empty:
        missing_info = pd.concat([missing_info, incorrect_readings[['grand_prix', 'location', 'driver', 'session_uid']]], ignore_index=True)

    return correct_readings, missing_info


In [6]:
def get_session_data(year, race_weekend, session_name):
    """
    Get throttle and braking data for all drivers in a specific session.

    Parameters:
    - year: int, season year (e.g., 2023)
    - race_weekend: string, track location (e.g., 'Suzuka', 'Monza')
    - session_name: string, session type ('FP1', 'FP2', 'FP3', 'SQ', 'SS', 'R')

    Returns:
    - full_throttle: DataFrame with driver telemetry metrics
    - missing_info: DataFrame with drivers who lacked usable data
    """
    session = ff1.get_session(year, race_weekend, session_name)
    session.load(telemetry=True, laps=True)

    full_throttle, missing_info = get_all_drivers_throttle_input(
        session,
        season=year,
        session_name=session_name
    )

    # Add session name to both outputs
    full_throttle['session'] = session_name
    missing_info['session'] = session_name

    return full_throttle, missing_info


In [7]:
def get_track_full_throttle(season, most_recent_practice=True):
    """
    Collect throttle/braking/tyre/weather info from all pre-qualifying sessions of the season.

    Parameters:
    - season: int (e.g., 2023)
    - most_recent_practice: bool, whether to fetch only the most recent weekend (default: True)

    Returns:
    - throttle_df: DataFrame with all collected driver telemetry metrics
    - missing_df: DataFrame with all drivers missing valid telemetry
    """
    schedule = ff1.get_event_schedule(season)
    season_races = schedule[
        (schedule.Session1DateUtc < datetime.utcnow()) &
        (schedule.EventFormat != 'testing')
    ]

    if most_recent_practice:
        races = season_races[season_races.RoundNumber == season_races.RoundNumber.max()]
    else:
        races = season_races.copy()

    throttle_rows = []
    missing_rows = []

    conventional_sessions = ['FP1', 'FP2', 'FP3']

    for _, row in races.iterrows():
        race_location = row['Location']
        format_type = row['EventFormat']
        print(f"Processing {race_location} ({format_type})...")

        if format_type == 'conventional':
            sessions = conventional_sessions
        else:
            sessions = ['FP1']

        for session_name in sessions:
            try:
                full_throttle, missing_info = get_session_data(season, race_location, session_name)
                throttle_rows.append(full_throttle)
                missing_rows.append(missing_info)
            except Exception as e:
                print(f"⚠️ Skipped {race_location} {session_name}: {e}")

    throttle_df = pd.concat(throttle_rows, ignore_index=True) if throttle_rows else pd.DataFrame()
    missing_df = pd.concat(missing_rows, ignore_index=True) if missing_rows else pd.DataFrame()

    return throttle_df, missing_df


In [8]:
@lru_cache(maxsize=None)
def get_elevation(latitude, longitude):
    """ 
    Get the elevation of a single point based on coordinates.
    
    Parameters:
    - latitude
    - longitude

    Returns single value - altitude above sea level in meters.
    """
    url = f"https://api.open-elevation.com/api/v1/lookup?locations={latitude},{longitude}"
    
    response = requests.get(url)
    data = response.json()
    elevation = data["results"][0]["elevation"]

    return elevation

In [9]:
def get_circuits(season):
    """
    Get main geolocation info of the tracks:
    - latitude
    - longitude
    - official circuit name
    - altitude above sea level

    Parameters:
    - season: int indicating the year/season of Formula 1

    Returns:
    - DataFrame with circuitName, location, latitude, longitude, and altitude
    """
    ergast = Ergast()
    racetracks = ergast.get_circuits(season)

    results = []

    for racetrack in racetracks.circuitName:
        try:
            row = racetracks[racetracks.circuitName == racetrack].iloc[0]
            latitude = row.iloc[3]
            longitude = row.iloc[4]
            locality = row.iloc[2] if len(row) > 2 else None

            altitude = get_elevation(latitude, longitude)

            results.append({
                'circuitName': racetrack,
                'location': locality,
                'lat': latitude,
                'lon': longitude,
                'altitude': altitude
            })

        except Exception as e:
            print(f"⚠️ Failed to get altitude for {racetrack}: {e}")
            continue

    return pd.DataFrame(results)


In [10]:
def get_circuit_corner_profile(session, low_thresh=100, med_thresh=170):
    """
    Analyze a session's fastest lap to classify corners by speed.

    Parameters:
    - session: loaded FastF1 session
    - low_thresh: threshold for slow corners (default: 100 km/h)
    - med_thresh: threshold for medium corners (default: 170 km/h)

    Returns:
    - dict with counts of slow/medium/fast corners and chicanes
    """
    try:
        lap = session.laps.pick_fastest()
        tel = lap.get_car_data().add_distance()

        # Find local speed minima
        tel['prev_speed'] = tel['Speed'].shift(1)
        tel['next_speed'] = tel['Speed'].shift(-1)
        tel['is_corner'] = (tel['Speed'] < tel['prev_speed']) & (tel['Speed'] < tel['next_speed'])

        corners = tel[tel['is_corner']].copy()
        corners['corner_type'] = pd.cut(
            corners['Speed'],
            bins=[0, low_thresh, med_thresh, 400],
            labels=['slow', 'medium', 'fast']
        )

        counts = corners['corner_type'].value_counts().to_dict()

        # Optional: detect chicanes (2 corners < 200m apart)
        corners['DistanceFromPrev'] = corners['Distance'].diff().fillna(9999)
        chicanes = (corners['DistanceFromPrev'] < 200).sum()

        return {
            'slow_corners': counts.get('slow', 0),
            'medium_corners': counts.get('medium', 0),
            'fast_corners': counts.get('fast', 0),
            'chicanes': chicanes
        }

    except Exception as e:
        print(f"⚠️ Corner profile failed: {e}")
        return {
            'slow_corners': None,
            'medium_corners': None,
            'fast_corners': None,
            'chicanes': None
        }


In [11]:
def get_all_circuits(start_year=2020, end_year=2025):
    all_rows = []
    for year in range(start_year, end_year + 1):
        df = get_circuits(year)
        all_rows.append(df)
    
    full_df = pd.concat(all_rows, ignore_index=True)
    
    # Keep only the first unique occurrence per circuit
    deduped = full_df.drop_duplicates(subset=['circuitName'], keep='first').reset_index(drop=True)
    
    return deduped


In [12]:
circuit_metadata = get_all_circuits(2020, 2025)

In [13]:
circuit_metadata.loc[circuit_metadata.circuitName=='Suzuka Circuit']

,circuitName,location,lat,lon,altitude
26,Suzuka Circuit,Suzuka Circuit,34.8431,136.541,28.0


In [14]:
def extract_track_metrics(session):
    """
    Extracts average speed, top speed, throttle zone ratios, and braking zone count.

    Parameters:
    - session: FastF1 session object (must be loaded)

    Returns:
    - dict of key track-level metrics
    """
    try:
        lap = session.laps.pick_fastest()
        telemetry = lap.get_car_data().add_distance()

        telemetry['delta_speed'] = telemetry['Speed'].diff()
        heavy_brakes = telemetry['delta_speed'] < -30
        braking_events = heavy_brakes.sum()

        low = (telemetry.Speed < 120).mean()
        med = ((telemetry.Speed >= 120) & (telemetry.Speed < 200)).mean()
        high = (telemetry.Speed >= 200).mean()

        return {
            'avg_speed': telemetry['Speed'].mean(),
            'top_speed': telemetry['Speed'].max(),
            'braking_events': braking_events,
            'low_pct': low,
            'med_pct': med,
            'high_pct': high
        }
    except Exception as e:
        print(f"⚠️ Failed to extract metrics: {e}")
        return None


In [15]:
def build_circuit_profile_df(start_year=2020, end_year=2025):
    """
    Builds a DataFrame with circuit-level metrics for FP1 sessions across seasons.

    Parameters:
    - start_year: int, first season to include
    - end_year: int, last season to include

    Returns:
    - DataFrame with circuit performance and layout characteristics, including altitude and DRS zones
    """
    
    # Get all altitudes per circuit per year
    circuit_metadata = pd.concat(
        [get_circuits(year).assign(year=year) for year in range(start_year, end_year + 1)],
        ignore_index=True
    )

    records = []

    for year in range(start_year, end_year + 1):
        schedule = ff1.get_event_schedule(year)
        races = schedule[
            (schedule.Session1DateUtc < datetime.utcnow()) &
            (schedule.EventFormat != 'testing')
        ]

        for _, row in races.iterrows():
            location = row['Location']
            event_name = row['EventName']

            try:
                print(f"🔍 Processing {year} {location} FP1")
                session = ff1.get_session(year, location, 'FP1')
                session.load(telemetry=True, laps=True)

                track_length = getattr(session, 'track_length', None)
                track_length = track_length if track_length else np.nan

                # Get DRS info
                try:
                    drs_data = session.drs_zones
                    num_drs_zones = len(drs_data) if drs_data else 0
                    drs_total_len = sum(z.get('zoneLength', 0) for z in drs_data) if drs_data else 0
                    drs_pct = drs_total_len / track_length if track_length and drs_total_len else np.nan
                except AttributeError:
                    num_drs_zones = 0
                    drs_total_len = 0
                    drs_pct = np.nan

                # Extract metrics
                track_metrics = extract_track_metrics(session)
                corner_data = get_circuit_corner_profile(session)

                # Get altitude dynamically from circuit metadata
                alt_lookup = circuit_metadata[
                    (circuit_metadata['location'] == location) |
                    (circuit_metadata['circuitName'].str.contains(location))
                ]
                altitude = alt_lookup['altitude'].values[0] if not alt_lookup.empty else None

                if track_metrics:
                    record = {
                        'year': year,
                        'location': location,
                        'event': event_name,
                        'real_altitude': altitude,
                        'num_drs_zones': num_drs_zones,
                        'drs_total_len_m': drs_total_len,
                        'drs_pct_of_lap': drs_pct,
                        **track_metrics,
                        **corner_data
                    }
                    records.append(record)

            except Exception as e:
                print(f"⚠️ Skipped {year} {location}: {e}")

    return pd.DataFrame(records)


# Work in progress

1. quali pace
2. race pace
3. breaking
4. when to query the data?
    - the time of the race is not consistent
    - on sprint race weekends the quali takes place on Friday
    - US/Asia/Australia races happen at different times

In [16]:
schedule = ff1.get_event_schedule(2025)

In [17]:
schedule.loc[schedule.EventFormat!='testing',

        ['RoundNumber',
          'Country',
          'Location',
          'EventName',
          'EventFormat',
          'Session1',
          'Session1DateUtc',
          'Session2',
          'Session2DateUtc',
          'Session3',
          'Session3DateUtc',
          'Session4',
          'Session4DateUtc',
          'Session5',
          'Session5DateUtc',
          ]]

,RoundNumber,Country,Location,EventName,EventFormat,Session1,Session1DateUtc,Session2,Session2DateUtc,Session3,Session3DateUtc,Session4,Session4DateUtc,Session5,Session5DateUtc
1,1,Australia,Melbourne,Australian Grand Prix,conventional,Practice 1,2025-03-14 01:30:00,Practice 2,2025-03-14 05:00:00,Practice 3,2025-03-15 01:30:00,Qualifying,2025-03-15 05:00:00,Race,2025-03-16 04:00:00
2,2,China,Shanghai,Chinese Grand Prix,sprint_qualifying,Practice 1,2025-03-21 03:30:00,Sprint Qualifying,2025-03-21 07:30:00,Sprint,2025-03-22 03:00:00,Qualifying,2025-03-22 07:00:00,Race,2025-03-23 07:00:00
3,3,Japan,Suzuka,Japanese Grand Prix,conventional,Practice 1,2025-04-04 02:30:00,Practice 2,2025-04-04 06:00:00,Practice 3,2025-04-05 02:30:00,Qualifying,2025-04-05 06:00:00,Race,2025-04-06 05:00:00
4,4,Bahrain,Sakhir,Bahrain Grand Prix,conventional,Practice 1,2025-04-11 11:30:00,Practice 2,2025-04-11 15:00:00,Practice 3,2025-04-12 12:30:00,Qualifying,2025-04-12 16:00:00,Race,2025-04-13 15:00:00
5,5,Saudi Arabia,Jeddah,Saudi Arabian Grand Prix,conventional,Practice 1,2025-04-18 13:30:00,Practice 2,2025-04-18 17:00:00,Practice 3,2025-04-19 13:30:00,Qualifying,2025-04-19 17:00:00,Race,2025-04-20 17:00:00
6,6,United States,Miami,Miami Grand Prix,sprint_qualifying,Practice 1,2025-05-02 16:30:00,Sprint Qualifying,2025-05-02 20:30:00,Sprint,2025-05-03 16:00:00,Qualifying,2025-05-03 20:00:00,Race,2025-05-04 20:00:00
7,7,Italy,Imola,Emilia Romagna Grand Prix,conventional,Practice 1,2025-05-16 11:30:00,Practice 2,2025-05-16 15:00:00,Practice 3,2025-05-17 10:30:00,Qualifying,2025-05-17 14:00:00,Race,2025-05-18 13:00:00
8,8,Monaco,Monaco,Monaco Grand Prix,conventional,Practice 1,2025-05-23 11:30:00,Practice 2,2025-05-23 15:00:00,Practice 3,2025-05-24 10:30:00,Qualifying,2025-05-24 14:00:00,Race,2025-05-25 13:00:00
9,9,Spain,Barcelona,Spanish Grand Prix,conventional,Practice 1,2025-05-30 11:30:00,Practice 2,2025-05-30 15:00:00,Practice 3,2025-05-31 10:30:00,Qualifying,2025-05-31 14:00:00,Race,2025-06-01 13:00:00
10,10,Canada,Montréal,Canadian Grand Prix,conventional,Practice 1,2025-06-13 17:30:00,Practice 2,2025-06-13 21:00:00,Practice 3,2025-06-14 16:30:00,Qualifying,2025-06-14 20:00:00,Race,2025-06-15 18:00:00


In [ ]:
circuit_info=build_circuit_profile_df(2020,2025)

events      WARNING 	Correcting user input 'Spielberg' to 'Styrian Grand Prix'
core           INFO 	Loading data for Styrian Grand Prix - Practice 1 [v3.5.3]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info
req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...


🔍 Processing 2020 Spielberg FP1


req            INFO 	Using cached data for car_data
req            INFO 	Using cached data for position_data
req            INFO 	Using cached data for weather_data
req            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['3', '4', '5', '6', '7', '8', '10', '11', '16', '18', '20', '23', '26', '31', '33', '40', '44', '55', '77', '88']
events      WARNING 	Correcting user input 'Spielberg' to 'Styrian Grand Prix'
core           INFO 	Loading data for Styrian Grand Prix - Practice 1 [v3.5.3]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info
req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...


🔍 Processing 2020 Spielberg FP1


req            INFO 	Using cached data for car_data
req            INFO 	Using cached data for position_data
req            INFO 	Using cached data for weather_data
req            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['3', '4', '5', '6', '7', '8', '10', '11', '16', '18', '20', '23', '26', '31', '33', '40', '44', '55', '77', '88']
core           INFO 	Loading data for Hungarian Grand Prix - Practice 1 [v3.5.3]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info
req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
core        WARNING 	No lap data for driver 10


...

req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for driver_info. Loading data...
_api           INFO 	Fetching driver list...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for session_status_data. Loading data...
_api           INFO 	Fetching session status data...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for track_status_data. Loading data...
_api           INFO 	Fetching track status data...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for _extended_timing_data. Loading data...
_api           INFO 	Fetching timing data...
_api           INFO 	Parsing timing data...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for timing_app_data. Loading data...
_api           INFO 	Fetching timing app data...
req            INFO 	Data has been written to

🔍 Processing 2022 Mexico City FP1


req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for driver_info. Loading data...
_api           INFO 	Fetching driver list...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for session_status_data. Loading data...
_api           INFO 	Fetching session status data...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for track_status_data. Loading data...
_api           INFO 	Fetching track status data...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for _extended_timing_data. Loading data...
_api           INFO 	Fetching timing data...
_api           INFO 	Parsing timing data...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for timing_app_data. Loading data...
_api           INFO 	Fetching timing app data...
req            INFO 	Data has been written to

🔍 Processing 2022 São Paulo FP1


req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for driver_info. Loading data...
_api           INFO 	Fetching driver list...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for session_status_data. Loading data...
_api           INFO 	Fetching session status data...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for track_status_data. Loading data...
_api           INFO 	Fetching track status data...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for _extended_timing_data. Loading data...
_api           INFO 	Fetching timing data...
_api           INFO 	Parsing timing data...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for timing_app_data. Loading data...
_api           INFO 	Fetching timing app data...
req            INFO 	Data has been written to

In [ ]:
circuit_info

In [ ]:
def create_corners_info(location, slow, medium, fast):
    return pd.DataFrame([{
        'Location': location,
        'slow':slow,
        'medium':medium,
        'fast':fast
        }])

def create_pirelli_tyre_info(location, traction, evolution, lateral_load, abrasion, braking, grip, tyre_stress, downforce):
    return pd.DataFrame([{
        'Location': location,
        'traction': traction,
        'evolution': evolution,
        'lateral_load': lateral_load,
        'abrasion': abrasion,
        'braking': braking,
        'grip': grip,
        'tyre_stress': tyre_stress,
        'downforce':downforce
        }])


In [ ]:
bahrain_corners = create_corners_info('Sakhir',4,4,7)
jeddah_corners = create_corners_info('Jeddah',4,4,7)
melbourne_corners = create_corners_info('Melbourne', 3, 5, 6)
baku_corners = create_corners_info('Baku',11,3,6)
miami_corners = create_corners_info('Miami', 7,5,7)
monaco_corners = create_corners_info('Monaco', 9,5,5)
barcelona_corners = create_corners_info('Barcelona', 2,6,6) #2023
barcelona_corners = create_corners_info('Barcelona', 4,8,4) #2022
montreal_corners = create_corners_info('Montreal', 3,8,3)
spielberg_corners = create_corners_info('Spielberg', 2,3,5)
silverstone_corners = create_corners_info('Silverstone', 4,4,10)
budapest_corners = create_corners_info('Budapest', 3,7,4)
spa_corners = create_corners_info('Spa', 3,8,8)
imola_corners = create_corners_info('Imola', 5,3,11)
abu_dhabi_pirelli = create_corners_info('Abu Dhabi',3,6,7)
las_vegas_corners = create_corners_info('Las Vegas', 7,4,6)
mexico_corners = create_corners_info('Mexico City', 7,6,4)
sao_paulo_corners = create_corners_info('Sao Paulo', 1,6,8)
suzuka_corners = create_corners_info('Suzuka', 3,4,11)
qatar_corners = create_corners_info('Qatar', 1,4,11)
austin_corners = create_corners_info('Austin', 7,5,8)
zandvoort_corners = create_corners_info('Zandvoort', 2,6,6)
monza_corners = create_corners_info('Monza', 2,5,4)
singapore_corners = create_corners_info('Singapore', 10,8,5) #2022
singapore_corners = create_corners_info('Singapore', 7,7,5) #2023

corners = pd.concat([bahrain_corners,
           jeddah_corners,
           melbourne_corners,
           baku_corners,
           miami_corners,
           monaco_corners,
           barcelona_corners,
           montreal_corners,
           spielberg_corners,
           silverstone_corners ,
           budapest_corners ,
           spa_corners ,
           imola_corners ,
           abu_dhabi_pirelli ,
           las_vegas_corners ,
           mexico_corners,
           sao_paulo_corners ,
           suzuka_corners ,
           qatar_corners ,
           austin_corners,
           zandvoort_corners,
           monza_corners,
           singapore_corners,
           ], 
           ignore_index=True
           )


In [ ]:
bahrain_pirelli = create_pirelli_tyre_info('Sakhir', 4, 4, 3, 5, 4, 3, 3, 3)
jeddah_pirelli = create_pirelli_tyre_info('Jeddah', 2, 3, 4, 2, 2, 3, 3, 2)
melbourne_pirelli = create_pirelli_tyre_info('Melbourne', 2, 4, 3, 2, 2, 3, 3, 3)
baku_pirelli = create_pirelli_tyre_info('Baku', 5, 5, 1, 1, 4, 1, 3, 1)
miami_pirelli = create_pirelli_tyre_info('Miami',3,5,3,2,3,3,3,2)
monaco_pirelli = create_pirelli_tyre_info('Monaco', 5,5,1,1,2,1,1,5)
barcelona_pirelli = create_pirelli_tyre_info('Barcelona', 3,3,5,4,3,3,5,4) #2023
barcelona_pirelli = create_pirelli_tyre_info('Barcelona', 3,3,4,4,3,3,4,4) #2022
montreal_pirelli = create_pirelli_tyre_info('Montreal', 5,5,1,2,5,1,3,1)

spielberg_pirelli = create_pirelli_tyre_info('Spielberg', 2,3,5)
silverstone_pirelli = create_pirelli_tyre_info('Silverstone', 4,4,10)
budapest_pirelli = create_pirelli_tyre_info('Budapest', 3,7,4)
spa_pirelli = create_pirelli_tyre_info('Spa', 3,8,8)
imola_pirelli = create_pirelli_tyre_info('Imola', 5,3,11)
abu_dhabi_pirelli = create_pirelli_tyre_info('Abu Dhabi',3,6,7)
las_vegas_pirelli = create_pirelli_tyre_info('Las Vegas', 7,4,6)
mexico_pirelli = create_pirelli_tyre_info('Mexico City', 7,6,4)
sao_paulo_pirelli = create_pirelli_tyre_info('Sao Paulo', 1,6,8)
suzuka_pirelli = create_pirelli_tyre_info('Suzuka', 3,4,11)
qatar_pirelli = create_pirelli_tyre_info('Qatar', 1,4,11)
austin_pirelli = create_pirelli_tyre_info('Austin', 7,5,8)
zandvoort_pirelli = create_pirelli_tyre_info('Zandvoort', 2,6,6)
monza_pirelli = create_pirelli_tyre_info('Monza', 2,5,4)
singapore_pirelli = create_pirelli_tyre_info('Singapore', 10,8,5) #2022
singapore_pirelli = create_pirelli_tyre_info('Singapore', 7,7,5) #2023


In [ ]:
pirelli = pd.concat([
    bahrain_pirelli,
    jeddah_pirelli,
    melbourne_pirelli,
    baku_pirelli,
    miami_pirelli ,
    monaco_pirelli ,
    barcelona_pirelli,
    montreal_pirelli ,
    ], 
    ignore_index=True
    )